<a href="https://colab.research.google.com/github/RafsanJany-44/Research_Sleep_Stage_Classification/blob/main/LIME.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

data = 'https://raw.githubusercontent.com/RafsanJany-44/Thesis_Project/main/All_DATA/without_sn_and_Epoch/EEG_HMC.csv'
dataset = pd.read_csv(data)
X = dataset.iloc[:, 1:]
y = dataset.iloc[:, 0]


In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [3]:
X_train.shape, X_test.shape

((72662, 75), (35789, 75))

StandardScaler() returns a numpy array. For better understandings we convert that in to pandas dataframe. StandardScaler() returns a numpy array. For better understanding we convert that in to pandas dataframe. And in separation for X and y, we remove the <b><i>[.values]</i></b>.

In [4]:
X_train2=X_train
X_test2=X_test

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train =sc.fit_transform(X_train)

X_test =sc.fit_transform(X_test)


X_train = pd.DataFrame(X_train, index=X_train2.index, columns=X_train2.columns)
X_test = pd.DataFrame(X_test, index=X_test2.index, columns=X_test2.columns)


**Tuning**

KNN

In [5]:
from sklearn.neighbors import KNeighborsClassifier
KNN_class = KNeighborsClassifier(n_neighbors=7)
KNN_class.fit(X_train, y_train)
y_pred_KNN=KNN_class.predict(X_test)

Radomforest

In [16]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=125, random_state=0)
rf.fit(X_train, y_train)
y_pred_RF=rf.predict(X_test)

xgboost

In [7]:
import xgboost as xgb
xgb_classifier = xgb.XGBClassifier(max_depth=14)
xgb_classifier.fit(X_train.values, y_train.values)  
y_pred_XG = xgb_classifier.predict(X_test.values)

Accurecy

In [8]:
from sklearn.metrics import accuracy_score
print("Accurecy KNN: ",accuracy_score(y_test, y_pred_KNN))
print("Accurecy RF: ",accuracy_score(y_test, y_pred_RF))
print("Accurecy XG: ",accuracy_score(y_test, y_pred_XG))


Accurecy KNN:  0.7467657660174915
Accurecy RF:  0.7600380005029478
Accurecy XG:  0.7751543770432255


#LIME

For further usage and Rearch perpose

In [ ]:
!pip install lime

In [ ]:
X_test.iloc[8]

In [ ]:
ex_specie = np.array(X_test.iloc[10]).reshape(1,-1)

In [ ]:
print(X_test.iloc[[10]])

In [ ]:
print("Prediction for KNN: ",KNN_class.predict(ex_specie)[0])
print("Prediction for RF: ",rf.predict(ex_specie)[0])
print("Prediction for XG: ",xgb_classifier.predict(np.array(X_train.iloc[[10]]))[0])
print("Prediction Accual: ",list(y_test.iloc[[10]])[0])

In [ ]:
print("Prediction Accual: ",list(y_test.iloc[[10]])[0])

In [ ]:
import lime
from lime import lime_tabular
explainer = lime_tabular.LimeTabularExplainer(
    training_data=np.array(X_train),
    feature_names=list(dataset.iloc[:0,1:]),
    class_names=['N1','N2','N3','R','W'],
    mode='classification'
)

In [ ]:
exp_KNN = explainer.explain_instance(X_test.iloc[10],
                                 KNN_class.predict_proba,
                                 num_features=6,
                                 top_labels=5)

#exp_KNN.show_in_notebook(show_table=True, show_all=False)

In [ ]:
exp_KNN.save_to_file("KNN.html")

In [ ]:
exp_rf = explainer.explain_instance(X_test.iloc[10],
                                 rf.predict_proba,
                                 num_features=6,
                                 top_labels=5)

#exp_rf.show_in_notebook(show_table=True, show_all=False)

In [ ]:
exp_rf.save_to_file("RandomForest.html")

In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(X_train.values,
                                                   feature_names=list(dataset.iloc[:0,1:]),
                                                   class_names=['N1','N2','N3','R','W'])

In [ ]:
exp_xg = explainer.explain_instance(X_train.iloc[10, :].values,
                                 xgb_classifier.predict_proba,
                                 num_features=6,
                                 top_labels=5)

#exp_xg.show_in_notebook(show_table=True, show_all=False)

In [ ]:
exp_xg.save_to_file("XG_boost.html")



---





---



---



#Reaserch Execution
For only this reasearch

Randomly chossing Rows

In [17]:
from sklearn.utils.extmath import row_norms
from tqdm import tqdm
import random
all=[]
none=[]
onlyRF=[]
onlyXG=[]
onlyKNN=[]

RFKNN=[]
RFXG=[]
KNNXG=[]


for i in tqdm(range(0,100)):
  print("For:",i)
  ex_specie = np.array(X_test.iloc[i]).reshape(1,-1)
  knn=KNN_class.predict(ex_specie)[0]
  ranf=rf.predict(ex_specie)[0]
  xg=xgb_classifier.predict(np.array(X_train.iloc[[i]]))[0]
  y=y_test.iloc[i]
  print("Prediction for KNN: ",knn)
  print("Prediction for RF: ",ranf)
  print("Prediction for XG: ",xg)
  print("           Actual: ",y)
  
  if y==knn and y==ranf and y==xg:
    all.append(i)
  
  elif y==knn and y!=ranf and y!=xg:
    onlyKNN.append(i)
  
  elif y!=knn and y==ranf and y!=xg:
    onlyRF.append(i)
  
  elif y!=knn and y!= ranf and y==xg:
    onlyXG.append(i)
  
  elif y==knn and y==ranf and y!=xg:
    RFKNN.append(i)
  
  elif y!=knn and y==ranf and y==xg:
    RFXG.append(i)
  
  elif y==knn and y!=ranf and y==xg:
    KNNXG.append(i)
  
  else:
    none.append(i)

temp_rows=[all,none,onlyRF,onlyXG,onlyKNN,RFKNN,RFXG,KNNXG]
sec=["ALL","NONE","ONLY_RF","ONLY_XG","ONLY_KNN","RF_KNN","RF_XG","KNN_XG"]
rows=[]

for i in temp_rows:
  rows.append(random.choice(i))

  0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but KNe

For: 0
Prediction for KNN:  R
Prediction for RF:  RandomForestClassifier(n_estimators=125, random_state=0)
Prediction for XG:  N2
           Actual:  R
For: 1
Prediction for KNN:  R
Prediction for RF:  RandomForestClassifier(n_estimators=125, random_state=0)
Prediction for XG:  N2
           Actual:  R
For: 2
Prediction for KNN:  N2
Prediction for RF:  RandomForestClassifier(n_estimators=125, random_state=0)
Prediction for XG:  N1
           Actual:  N2
For: 3
Prediction for KNN:  N2
Prediction for RF:  RandomForestClassifier(n_estimators=125, random_state=0)
Prediction for XG:  W
           Actual:  N2
For: 

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"
  6%|▌         | 6/100 [00:00<00:04, 19.52it/s]/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names,

4
Prediction for KNN:  N1
Prediction for RF:  RandomForestClassifier(n_estimators=125, random_state=0)
Prediction for XG:  W
           Actual:  W
For: 5
Prediction for KNN:  N3
Prediction for RF:  RandomForestClassifier(n_estimators=125, random_state=0)
Prediction for XG:  W
           Actual:  N3
For: 6
Prediction for KNN:  N2
Prediction for RF:  RandomForestClassifier(n_estimators=125, random_state=0)
Prediction for XG:  W
           Actual:  W
For: 7
Prediction for KNN:  N2
Prediction for RF:  RandomForestClassifier(n_estimators=125, random_state=0)
Prediction for XG:  N2
           Actual:  N1
For: 8


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"
 10%|█         | 10/100 [00:00<00:04, 19.06it/s]/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names

...

R
           Actual:  N2
For: 92
Prediction for KNN:  N1
Prediction for RF:  RandomForestClassifier(n_estimators=125, random_state=0)
Prediction for XG:  N2
           Actual:  N2
For: 93
Prediction for KNN:  W
Prediction for RF:  RandomForestClassifier(n_estimators=125, random_state=0)
Prediction for XG:  N2
           Actual:  W
For: 94
Prediction for KNN:  N3
Prediction for RF:  RandomForestClassifier(n_estimators=125, random_state=0)
Prediction for XG:  W
           Actual:  N3
For: 95


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"
 96%|█████████▌| 96/100 [00:06<00:00, 14.84it/s]/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names

Prediction for KNN:  N2
Prediction for RF:  RandomForestClassifier(n_estimators=125, random_state=0)
Prediction for XG:  W
           Actual:  N2
For: 96
Prediction for KNN:  R
Prediction for RF:  RandomForestClassifier(n_estimators=125, random_state=0)
Prediction for XG:  R
           Actual:  N1
For: 97
Prediction for KNN:  R
Prediction for RF:  RandomForestClassifier(n_estimators=125, random_state=0)
Prediction for XG:  W
           Actual:  R
For: 98


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"
100%|██████████| 100/100 [00:06<00:00, 14.64it/s]

Prediction for KNN:  N2
Prediction for RF:  RandomForestClassifier(n_estimators=125, random_state=0)
Prediction for XG:  N3
           Actual:  N2
For: 99
Prediction for KNN:  N2
Prediction for RF:  RandomForestClassifier(n_estimators=125, random_state=0)
Prediction for XG:  N2
           Actual:  N1


In [18]:
print(rows)
print(sec)

[73, 58, 84, 83, 43, 3, 67, 48]
['ALL', 'NONE', 'ONLY_RF', 'ONLY_XG', 'ONLY_KNN', 'RF_KNN', 'RF_XG', 'KNN_XG']


In [9]:
!pip install lime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 275 kB 27.6 MB/s 
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283857 sha256=657cba6b6f4a6ef82b86f8e256c674e81e2cc8df7f75b85ae19171984541131e
  Stored in directory: /root/.cache/pip/wheels/ca/cb/e5/ac701e12d365a08917bf4c6171c0961bc880a8181359c66aa7
Successfully built lime


In [20]:
from tqdm import tqdm
import lime
from lime import lime_tabular

feat_num=5
r=1

for row in tqdm(rows):
  print("___________________________________________________________________")
  print("Round ",r)
  print("___________________________________________________________________")
  ex_specie = np.array(X_test.iloc[row]).reshape(1,-1)

  print("Prediction for KNN: ",KNN_class.predict(ex_specie)[0])
  print("Prediction for RF: ",rf.predict(ex_specie)[0])
  print("Prediction for XG: ",xgb_classifier.predict(np.array(X_train.iloc[[row]]))[0])
  print("Prediction Accual: ",list(y_test.iloc[[row]])[0])

  print("___________________________________________________________________")


  cls_name=['N1','N2','N3','R','W']


  explainer_1 = lime_tabular.LimeTabularExplainer(
      training_data=np.array(X_train),
      feature_names=list(dataset.iloc[:0,1:]),
      class_names=cls_name,
      mode='classification'
  )


  explainer_2 = lime.lime_tabular.LimeTabularExplainer(
      training_data=X_train.values,
      feature_names=list(dataset.iloc[:0,1:]),
      class_names=cls_name,
  )


  exp_KNN = explainer_1.explain_instance(X_test.iloc[row],
                                  KNN_class.predict_proba,
                                  num_features=feat_num,
                                  top_labels=len(cls_name,))



  exp_rf = explainer_1.explain_instance(X_test.iloc[row],
                                  rf.predict_proba,
                                  num_features=feat_num,
                                  top_labels=len(cls_name,))



  exp_xg = explainer_2.explain_instance(X_train.iloc[row, :].values,
                                  xgb_classifier.predict_proba,
                                  num_features=feat_num,
                                  top_labels=len(cls_name,))





  print("___________________________________________________________________")
  print("___________________________________________________________________")




  exp_KNN.save_to_file('/content/ResultHTML/'+sec[rows.index(row)]+"___"+str(row)+"___KNN_.html")
  print("HTML file SAVED for KNN, row number=",row)

  exp_rf.save_to_file('/content/ResultHTML/'+sec[rows.index(row)]+"___"+str(row)+"___RandomForest_.html")
  print("HTML file SAVED for RF, row number=",row)

  exp_xg.save_to_file('/content/ResultHTML/'+sec[rows.index(row)]+"___"+str(row)+"___XG_boost_.html")
  print("HTML file SAVED for XG, row number=",row)
  
  print("___________________________________________________________________")
  print("___________________________________________________________________")
  r+=1

  0%|          | 0/8 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"


___________________________________________________________________
Round  1
___________________________________________________________________
Prediction for KNN:  N2
Prediction for RF:  N2
Prediction for XG:  N2
Prediction Accual:  N2
___________________________________________________________________


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"
 12%|█▎        | 1/8 [00:42<04:56, 42.40s/it]

___________________________________________________________________
___________________________________________________________________
HTML file SAVED for KNN, row number= 73
HTML file SAVED for RF, row number= 73
HTML file SAVED for XG, row number= 73
___________________________________________________________________
___________________________________________________________________
___________________________________________________________________
Round  2
___________________________________________________________________
Prediction for KNN:  W
Prediction for RF:  W
Prediction for XG:  N2
Prediction Accual:  N1
___________________________________________________________________


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"
 25%|██▌       | 2/8 [01:23<04:09, 41.56s/it]

...

___________________________________________________________________
___________________________________________________________________
HTML file SAVED for KNN, row number= 3
HTML file SAVED for RF, row number= 3
HTML file SAVED for XG, row number= 3
___________________________________________________________________
___________________________________________________________________
___________________________________________________________________
Round  7
___________________________________________________________________
Prediction for KNN:  N3
Prediction for RF:  N2
Prediction for XG:  N2
Prediction Accual:  N2
___________________________________________________________________


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"
 88%|████████▊ | 7/8 [04:47<00:40, 40.90s/it]

___________________________________________________________________
___________________________________________________________________
HTML file SAVED for KNN, row number= 67
HTML file SAVED for RF, row number= 67
HTML file SAVED for XG, row number= 67
___________________________________________________________________
___________________________________________________________________
___________________________________________________________________
Round  8
___________________________________________________________________
Prediction for KNN:  R
Prediction for RF:  W
Prediction for XG:  R
Prediction Accual:  R
___________________________________________________________________


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"
100%|██████████| 8/8 [05:28<00:00, 41.02s/it]

___________________________________________________________________
___________________________________________________________________
HTML file SAVED for KNN, row number= 48
HTML file SAVED for RF, row number= 48
HTML file SAVED for XG, row number= 48
___________________________________________________________________
___________________________________________________________________


In [19]:
print(rows)
print(sec)

[73, 58, 84, 83, 43, 3, 67, 48]
['ALL', 'NONE', 'ONLY_RF', 'ONLY_XG', 'ONLY_KNN', 'RF_KNN', 'RF_XG', 'KNN_XG']


In [12]:
print(sec[rows.index(49)])

RF_XG
